In [1]:
import pandas as pd #устанавливаем библиотеку для работы с таблицами

In [2]:
#создадим дата-фрейм для данных по мидиям в городе Санкт-Петербург
petersburg = {'СПБ': [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]}
petersburgframe = pd.DataFrame(petersburg)
petersburgframe

,СПБ
0,0.0974
1,0.1352
2,0.0817
3,0.1016
4,0.0968
5,0.1064
6,0.1050


In [3]:
#создадим дата-фрейм для данных по мидиям в городе Магадан
magadan = {'Магадан': [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]}
magadanframe = pd.DataFrame(magadan)
magadanframe

,Магадан
0,0.1033
1,0.0915
2,0.0781
3,0.0685
4,0.0677
5,0.0697
6,0.0764
7,0.0689


In [4]:
#объединим дата-фреймы Санкт-Петербурга и Магадана
midii = pd.concat([petersburgframe, magadanframe], axis=1)
display(midii)

,СПБ,Магадан
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,NaN,0.0689


In [5]:
#установим показатели для определения типа распределения
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

In [6]:
#установим уровень значимости
alpha = 0.05

In [7]:
from scipy.stats import shapiro #бибиотека для работы с тестом Шапиро-Уилка

Для определения типа распределения воспользуемся тестом Шапиро-Уилка

In [8]:
_, p = shapiro(petersburgframe)
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.242
Данные распределены нормально


In [9]:
_, p = shapiro(magadanframe)
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.036
Данные не распределены нормально (мы отвергаем H0)


Тест Шапиро-Уилка показал, что данные по Санкт-Петербургу распределены нормально, данные же по Магадану распределены ненормально.

In [10]:
from scipy.stats import normaltest #библиотека для работы с тестом Д'Агостино

Воспользуемся тестом Д'Агостино для определения типа распределения в городе Магадан. Для города Санкт-Петербурга он не подходит, поскольку имеет слишком маленькую выборку.

In [11]:
_, p = normaltest(magadanframe)
#normaltest возвращает двустороннюю вероятность для проверки гипотезы

print('p=%.3f' % p[0])

# Интерпретация 

if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.154
Данные распределены нормально


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1477: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


С другой стороны, тест Д'Агостино говорит, что данные по Магадану распределены нормально. Но тест Шапиро-Уилка лучше работает на небольших выборках. Поэтому примем, что данные по Магадану распределены скорее ненормально. Следовательно, воспользуемся непараметрической корреляцией Спирмана, которая вычисляет наличие корреляции между количественными переменными.

In [12]:
#устанавливаем библиотеку, работающую с корреляцией Спирмана
from scipy.stats import spearmanr

Тест Спирмана не работает с различным количеством данных в столбиках (7 в одном и 8 в другом). Поэтому нужно в столбике "Санкт-Петербург" убрать недостающее значение константой. В данном случае воспользуемся средним арифметическим, поскольку разброс данных не слишком большой (распеделение нормальное).

In [13]:
midii['СПБ'] = midii['СПБ'].fillna(midii['СПБ'].mean())

In [14]:
display(midii)

,СПБ,Магадан
0,0.097400,0.1033
1,0.135200,0.0915
2,0.081700,0.0781
3,0.101600,0.0685
4,0.096800,0.0677
5,0.106400,0.0697
6,0.105000,0.0764
7,0.103443,0.0689


In [15]:
corr, p = spearmanr(midii)

In [16]:
print(corr,p)

0.1666666666666667 0.693238811728395


In [17]:
if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутствии зависимости между переменными.")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.")

0.693238811728395 > 0.05. Мы не можем отвергнуть нулевую гипотезу об отсутствии зависимости между переменными.


Тест Спирмана показывает, что нет значимой связи между размерами мидий в Санкт-Петербурге и Магадане. Это значит, что с увеличением размеров мидий в Санкт-Петербурге, размеры мидий в Магадане не увеличиваются.

Для анализа средних размеров мидий в двух городах будем использовать тест Уилкоксона, посколько данные в нашем дата-сете непараметрические.

In [18]:
from scipy.stats import ttest_ind

In [19]:
H0 = 'Нет значимой разницы между размерами мидий в разных городах.'
Ha = 'Есть значимая разница между размерами мидий в разных городах.'

In [22]:
def t_test(midii):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(petersburgframe['СПБ'], magadanframe['Магадан'], equal_var=True)
    
    p = round(test_results[1],2)
    
    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(midii)


*** Результаты независимого T-теста ***
0.0
0.0 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между размерами мидий в разных городах.


Мы видим, что средний размер мидий в разных городах в среднем статистически различается.

In [28]:
print(petersburgframe['СПБ'].mean())#узнаем средний размер мидий в Санкт-Петербурге

0.10344285714285714


In [29]:
print(magadanframe['Магадан'].mean())#узнаем средний размер мидий в Магадане

0.0780125


Исходя из вышеперечисленных операций можно увидеть, что мидии в Санкт-Петербурге примерно в 1,25 раз больше, чем мидии в Магадане